**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import cv2
import json

try:
    import skvideo.io
except ImportError:
    !pip install scikit-video
    import skvideo.io

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


     |████████████████████████████████| 2.3MB 2.7MB/s 


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}\left[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Function `act` uses an $\epsilon$-greedy strategy to select an action.
It selects a previously learned action with probability $1 - \epsilon$ and a random action with probability $\epsilon$.

The point of this $\epsilon$ is to enforce state exploration, so that the learning algorithm does not always choose a sub-optimal learned action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Array `position` is an almost zero-filled array of size (`grid_size+4`, `grid_size+4`). It equals zero almost everywhere except on borders, where it equals -1 (representing the walls), and on the current position of the rat (where it equals 1). 

Array `board` contains the rewards and penalties, respectively represented by good and poisonous cheese.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        r"""
            Choose a random action among the possible ones
        """
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def float_to_string(x):
    r"""
        Format a float without trailing zeros
        Found on https://stackoverflow.com/questions/2440692/formatting-floats-in-python-without-trailing-zeros
    """
    return ('%.3f'%x).rstrip('0').rstrip('.')

def test(agent,env,epochs,prefix=''):
    np.random.seed(0)

    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
        
        state = env.reset()

        game_over = False

        win, lose = 0, 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})".format(
            float_to_string(win),
            float_to_string(lose),
            float_to_string(score/e)))
    print('Final score: '+str(score/epochs))

In [65]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random{:d}.mp4'.format(epochs_test)))

Win/lose count 7/18. Average score (-11)
Win/lose count 7.5/14. Average score (-8.75)
Win/lose count 8/16. Average score (-8.5)
Win/lose count 7.5/12. Average score (-7.5)
Win/lose count 12.5/15. Average score (-6.5)
Win/lose count 7.5/8. Average score (-5.5)
Win/lose count 8/8. Average score (-4.714)
Win/lose count 9.5/13. Average score (-4.562)
Win/lose count 10/18. Average score (-4.944)
Win/lose count 14.5/16. Average score (-4.6)
Final score: -4.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




$$\begin{aligned}
    Q^\pi(s,a)
    &= E_{p^{\pi}}\left[\sum_{t = 0}^\infty \gamma^t r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
    &= r(s,a) + \gamma E_{p^\pi} \left[\sum_{t=1}^\infty \gamma^{t-1} r(s_t,a_t) | s_0=s,a_0=a \right] \\
    &= r(s,a) + \sum_{s', a'} \left( \gamma E_{p^\pi} \left[\sum_{t=0}^\infty \gamma^t r(s_{t+1} ,a_{t+1}) | s_0 = s, a_0 = a, s_1 = s', a_1 = a' \right] \right) \\
    &= r(s,a) + \sum_{s', a'} p^\pi(s',a') \left( \gamma E_{p^\pi} \left[\sum_{t=0}^\infty \gamma^t r(s_t, a_t) | s_0 = s', a_0 = a' \right] \right) \\
    &= r(s,a) + E_{(s',a') \sim p^\pi(\cdot|s,a)} \left[ \gamma \underbrace{E_{p^\pi} \left[\sum_{t=0}^\infty \gamma^t r(s_t, a_t) | s_0 = s', a_0 = a' \right]}_{Q^\pi(s',a')} \right] \\
    &= E_{(s',a')\sim p^\pi(.|s,a)} \left[ r(s,a)+\gamma Q^{\pi}(s',a') \right]
\end{aligned}$$

Then:
$$\begin{aligned}
    Q^*(s,a)
    &= \max_\pi Q^\pi(s,a) \\
    &= \max_\pi E_{(s',a')\sim \pi(.|s,a)} \left[ r(s,a)+\gamma Q^{\pi}(s',a')\right] \\
    &= E_{(s',a')\sim p^{\pi^*}(.|s,a)} \left[ r(s,a)+\gamma Q^{\pi^*}(s',a') \right] \\
    &= E_{s' \sim \pi^*(.|s,a)} \left[ r(s,a)+\gamma Q^*(s',a') \right]
\end{aligned}$$

Therefore, for all states $s$ and all actions $a$,
$$Q(s,a,\theta) = Q^*(s,a) \Rightarrow \mathcal{L}(\theta) = 0$$

Since $\mathcal{L}$ is continuous and convex, optimizing $\mathcal{L}$ should make $Q$ be more and more closer to $Q^*$. It is thus a plausible objective.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        self.memory = self.memory[-self.max_memory:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    np.random.seed(0)

    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0 or e == epoch:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, float_to_string(win), float_to_string(lose), float_to_string(win-lose)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size = 16, n_state=2, n_action=4):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of states
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[np.newaxis]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        s_arr, n_s_arr, a_arr, r_arr, game_over_arr = list(zip(*[self.memory.random_access() for _ in range(self.batch_size)]))
        input_states = np.array(s_arr)

        target_q = np.array(r_arr)[:,np.newaxis] + np.array(game_over_arr)[:,np.newaxis]*self.discount*self.model.predict(np.array(n_s_arr))

        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Dense(25, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(self.n_action, activation='relu'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [114]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train{}.mp4'.format(epochs_train)))

Epoch 001/050 | Loss 0.0753 | Win/lose count 7/7 (0)
Epoch 002/050 | Loss 0.0626 | Win/lose count 5/6 (-1)
Epoch 003/050 | Loss 0.0782 | Win/lose count 3.5/4 (-0.5)
Epoch 004/050 | Loss 0.0076 | Win/lose count 2.5/5 (-2.5)
Epoch 005/050 | Loss 0.0359 | Win/lose count 2.5/7 (-4.5)
Epoch 006/050 | Loss 0.0075 | Win/lose count 4.5/6 (-1.5)
Epoch 007/050 | Loss 0.0079 | Win/lose count 2/0 (2)
Epoch 008/050 | Loss 0.0395 | Win/lose count 1/2 (-1)
Epoch 009/050 | Loss 0.0000 | Win/lose count 2.5/0 (2.5)
Epoch 010/050 | Loss 0.0073 | Win/lose count 1.5/3 (-1.5)
Epoch 011/050 | Loss 0.0000 | Win/lose count 1.5/1 (0.5)
Epoch 012/050 | Loss 0.0078 | Win/lose count 2/6 (-4)
Epoch 013/050 | Loss 0.0388 | Win/lose count 3.5/1 (2.5)
Epoch 014/050 | Loss 0.0312 | Win/lose count 3/3 (0)
Epoch 015/050 | Loss 0.0469 | Win/lose count 3.5/2 (1.5)
Epoch 016/050 | Loss 0.0078 | Win/lose count 2.5/1 (1.5)
Epoch 017/050 | Loss 0.0535 | Win/lose count 2.5/6 (-3.5)
Epoch 018/050 | Loss 0.0156 | Win/lose count 3

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32,(3,3),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Dropout(0.3))
        model.add(Conv2D(16,(2,2),activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(self.n_action))
        
        model.compile(adam(lr=lr), "mse")
        self.model = model

In [116]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon=0.1, memory_size=1000, batch_size=32)
print(agent.model.summary())
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train{:d}.mp4'.format(epochs_train)))

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_109 (Conv2D)          (None, 3, 3, 32)          608       
_________________________________________________________________
dropout_109 (Dropout)        (None, 3, 3, 32)          0         
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 2, 2, 16)          2064      
_________________________________________________________________
dropout_110 (Dropout)        (None, 2, 2, 16)          0         
_________________________________________________________________
flatten_68 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_81 (Dense)             (None, 4)                 260       
Total params: 2,932
Trainable params: 2,932
Non-trainable params: 0
___________________________________________________

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [118]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size=32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size=32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/2. Average score (-0.5)
Win/lose count 3.5/3. Average score (0)
Win/lose count 3/3. Average score (0)
Win/lose count 3/7. Average score (-1)
Win/lose count 1.5/2. Average score (-0.9)
Win/lose count 2.5/4. Average score (-1)
Win/lose count 2.5/2. Average score (-0.786)
Win/lose count 4/2. Average score (-0.438)
Win/lose count 2/1. Average score (-0.278)
Win/lose count 3/2. Average score (-0.15)
Final score: -0.15
Test of the FC
Win/lose count 2.5/6. Average score (-3.5)
Win/lose count 4.5/4. Average score (-1.5)
Win/lose count 4/2. Average score (-0.333)
Win/lose count 3/4. Average score (-0.5)
Win/lose count 3/1. Average score (0)
Win/lose count 1/5. Average score (-0.667)
Win/lose count 4/2. Average score (-0.286)
Win/lose count 1.5/4. Average score (-0.562)
Win/lose count 6/6. Average score (-0.5)
Win/lose count 4.5/10. Average score (-1)
Final score: -1.0


In [119]:
HTML(display_videos('cnn_test{}.mp4'.format(epochs_test)))

In [120]:
HTML(display_videos('fc_test{}.mp4'.format(epochs_test)))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, decay_factor, prefix=''):
    np.random.seed(0)
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        agent.set_epsilon(decay_factor*agent.epsilon)

        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0 or e == epoch:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, float_to_string(win), float_to_string(lose), float_to_string(win-lose)))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, revisit_penalty=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        self.explored = np.zeros((grid_size,grid_size), dtype=np.int32)
        self.revisit_penalty = revisit_penalty

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            raise RuntimeError('Error: action not recognized')

        self.t = self.t + 1

        # compute reward (add a penalty during training step if the case has already been explored)
        reward = self.board[self.x, self.y] - (self.revisit_penalty*self.explored[self.x, self.y] if train else 0)

        # clean the case and add it to explored cases
        self.board[self.x, self.y] = 0
        self.explored[self.x, self.y] += 1

        game_over = self.t > self.max_time
        state = np.stack((
            self.explored,
            self.board,
            self.position),
        axis=-1)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus
        self.explored = np.zeros((self.grid_size, self.grid_size), dtype=np.int32)

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.stack((
            self.explored,
            self.board,
            self.position),
        axis=-1)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [138]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon=0.2, memory_size=1000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, 0.95, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore{:d}.mp4'.format(epochs_train)))

Epoch 001/030 | Loss 0.2022 | Win/lose count 6.5/71.5 (-65)
Epoch 002/030 | Loss 0.1646 | Win/lose count 13.5/42.1 (-28.6)
Epoch 003/030 | Loss 0.1240 | Win/lose count 10/47.7 (-37.7)
Epoch 004/030 | Loss 0.2288 | Win/lose count 12.5/48.8 (-36.3)
Epoch 005/030 | Loss 0.1445 | Win/lose count 7.5/52.4 (-44.9)
Epoch 006/030 | Loss 0.1010 | Win/lose count 13.5/46.1 (-32.6)
Epoch 007/030 | Loss 0.1560 | Win/lose count 7/45.3 (-38.3)
Epoch 008/030 | Loss 0.1265 | Win/lose count 9.5/47.7 (-38.2)
Epoch 009/030 | Loss 0.1647 | Win/lose count 9.5/67.1 (-57.6)
Epoch 010/030 | Loss 0.1156 | Win/lose count 7.5/57.4 (-49.9)
Epoch 011/030 | Loss 0.0963 | Win/lose count 13/37.5 (-24.5)
Epoch 012/030 | Loss 0.1485 | Win/lose count 8.5/73.1 (-64.6)
Epoch 013/030 | Loss 0.1238 | Win/lose count 6.5/66.1 (-59.6)
Epoch 014/030 | Loss 0.0944 | Win/lose count 7/49.4 (-42.4)
Epoch 015/030 | Loss 0.2140 | Win/lose count 8/47.4 (-39.4)
Epoch 016/030 | Loss 0.1441 | Win/lose count 4.5/70.7 (-66.2)
Epoch 017/030 |

In [140]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore{:d}.mp4'.format(epochs_test)))

Win/lose count 3.5/5. Average score (-1.5)
Win/lose count 3.5/3. Average score (-0.5)
Win/lose count 2.5/3. Average score (-0.5)
Win/lose count 3.5/7. Average score (-1.25)
Win/lose count 6.5/4. Average score (-0.5)
Win/lose count 5.5/4. Average score (-0.167)
Win/lose count 1.5/6. Average score (-0.786)
Win/lose count 3.5/2. Average score (-0.5)
Win/lose count 3.5/8. Average score (-0.944)
Win/lose count 3/1. Average score (-0.65)
Final score: -0.65


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***